# 演習課題その4 L2R・検索結果多様化

## 必須課題(1) Ranking SVM
サンプルファイルとは別のテストデータを用意し，サンプルファイルの訓練データで学習したモデルをそのテストデータに適用してみよ． また，訓練データを観察し，どのような特徴量をもった文書が上位にランク付けされそうか自分で推測し，テストデータに対するランキング結果がその自分の推測と近いかどうか考察せよ．

In [5]:
import numpy as np
import pandas as pd

### 0. SVMRankモデルの学習
山本先生のNotebookの内容にしたがって，訓練データを利用したモデルの学習を行う．

In [1]:
# [START 訓練データの学習]
! ../bin/svm_rank/svm_rank_learn -c 0.03 ../data/svmrank_sample/train.dat ../data/svmrank_sample/model

Reading training examples...done
Training set properties: 5 features, 3 rankings, 12 examples
NOTE: Adjusted stopping criterion relative to maximum loss: eps=0.004667
Iter 1: ...*(NumConst=1, SV=1, CEps=4.6667, QPEps=0.0000)
Iter 2: ...*(NumConst=2, SV=1, CEps=4.2166, QPEps=0.0000)
Iter 3: ...(NumConst=2, SV=1, CEps=0.0000, QPEps=0.0000)
Final epsilon on KKT-Conditions: 0.00000
Upper bound on duality gap: -0.00000
Dual objective value: dval=0.13325
Primal objective value: pval=0.13325
Total number of constraints in final working set: 2 (of 2)
Number of iterations: 3
Number of calls to 'find_most_violated_constraint': 9
Number of SV: 1 
Norm of weight vector: |w|=0.11619
Value of slack variable (on working set): xi=4.21663
Value of slack variable (global): xi=4.21663
Norm of longest difference vector: ||Psi(x,y)-Psi(x,ybar)||=3.87313
Runtime in cpu-seconds: 0.00
Compacting linear model...done
Writing learned model...done


In [2]:
!cat ../data/svmrank_sample/model

SVM-light Version V6.20
0 # kernel type
3 # kernel parameter -d 
1 # kernel parameter -g 
1 # kernel parameter -s 
1 # kernel parameter -r 
empty# kernel parameter -u 
6 # highest feature index 
2 # number of training documents 
2 # number of support vectors plus 1 
0 # threshold b, each following line is a SV (starting with alpha*y)
1 1:0.099999994 2:-0.010000001 3:-0.049999997 4:-0.0010000003 5:0.029999999 #


In [3]:
! ../bin/svm_rank/svm_rank_classify ../data/svmrank_sample/test.dat ../data/svmrank_sample/model ../data/svmrank_sample/prediction

Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 0.0000
Zero/one-error on test set: 0.00% (1 correct, 0 incorrect, 1 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      0
Avg Swappedpairs Percent:   0.00


In [4]:
!cat ../data/svmrank_sample/prediction

0.12979999
0.08969999
0.02980000
-0.05020000


### 1. 別のテストデータの作成
`../data/svmrank_sample/train.dat`の内容から，どのような特徴量を持つ文書が上位にランキングされるかを考察する．
RankSVMでは，比較を行う2つの文書の**特徴量の差**を利用して選好の学習を行うという性質がある．
したがって，訓練データから`「上位の文書の特徴量」-「下位の文書の特徴量」`を計算し，
特徴量の符号の傾向を掴むことを考える．

以下は，**qid=1**の文書についてその傾向を把握するためのコードセルである．

In [7]:
# [START データの分析]
feat1 = [1, 0, 0, 0]
feat2 = [1, 0, 1, 0]
feat3 = [0, 1, 0, 1]
feat4 = [0.2, 0.1, 0.4, 0.3]
feat5 = [0, 1, 0, 0]
relevance = [3, 2, 1, 1]
train = pd.DataFrame({ "feat1": feat1, "feat2": feat2, "feat3": feat3, "feat4": feat4, "feat5": feat5, "relevance": relevance })
train.index = ["1A", "1B", "1C", "1D"]
train.head()

,feat1,feat2,feat3,feat4,feat5,relevance
1A,1,1,0,0.2,0,3
1B,0,0,1,0.1,1,2
1C,0,1,0,0.4,0,1
1D,0,0,1,0.3,0,1


In [12]:
for selection in [("1A", "1B"), ("1A", "1C"), ("1A", "1D"), ("1B", "1C"), ("1B", "1D")]:
    print("{} > {}".format(*selection))
    print((train.loc[selection[0]] - train.loc[selection[1]])[["feat1", "feat2", "feat3", "feat4", "feat5"]].values)

1A > 1B
[ 1.   1.  -1.   0.1 -1. ]
1A > 1C
[ 1.   0.   0.  -0.2  0. ]
1A > 1D
[ 1.   1.  -1.  -0.1  0. ]
1B > 1C
[ 0.  -1.   1.  -0.3  1. ]
1B > 1D
[ 0.   0.   0.  -0.2  1. ]


以上の表示結果から，**1次元目の特徴量の値が大きく，4次元目の特徴量の値が小さい**文書の方が，より上位に選ばれやすいといった傾向が掴める．

この考察結果を踏まえ，以下の二つのテストデータセットを作成した(それぞれ，test1.dat，test2.datとした)．
1. 上位の文書について，1次元目の特徴量が大きく，4次元目の特徴量が小さいようなデータセット
2. 1のデータセットとは傾向が逆(つまり1次元目の特徴量が小さく，4次元目の特徴量が大きいような文書が上位にランク付けされている)

In [13]:
!cat ../data/svmrank_sample/test1.dat

4 qid:10 1:3 2:0 3:1 4:0.3 5:0
3 qid:10 1:2 2:0 3:0 4:0.3 5:0
2 qid:10 1:0 2:0 3:0 4:0.1 5:0
1 qid:10 1:0 2:0 3:1 4:0.5 5:0


In [14]:
!cat ../data/svmrank_sample/test2.dat

4 qid:11 1:0 2:0 3:0 4:0.5 5:0
3 qid:11 1:1 2:0 3:0 4:0.8 5:0
2 qid:11 1:1 2:0 3:0 4:0.3 5:0
1 qid:11 1:1 2:0 3:0 4:0.1 5:0


### 2. テストデータへの適用
用意したテストデータについて，訓練データを利用して学習したモデルによる分類を行う．

In [15]:
! ../bin/svm_rank/svm_rank_classify ../data/svmrank_sample/test1.dat ../data/svmrank_sample/model ../data/svmrank_sample/prediction1

Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 0.0000
Zero/one-error on test set: 0.00% (1 correct, 0 incorrect, 1 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      0
Avg Swappedpairs Percent:   0.00


In [16]:
! ../bin/svm_rank/svm_rank_classify ../data/svmrank_sample/test2.dat ../data/svmrank_sample/model ../data/svmrank_sample/prediction2

Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 1.0000
Zero/one-error on test set: 100.00% (0 correct, 1 incorrect, 1 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      6
Avg Swappedpairs Percent: 100.00


### 3. テストデータに対する予測の表示と考察

In [17]:
! cat ../data/svmrank_sample/prediction1

0.24969999
0.19969999
-0.00010000
-0.05050000


In [18]:
! cat ../data/svmrank_sample/prediction2

-0.00050000
0.09919999
0.09969999
0.09989999


以上の結果から，1. のテストデータについては，全ての文書について実際の順位に対応するように適合度が予測されていることがわかる．
その一方で，2.のテストデータについては，全く逆の順番に対応するような適合度が予測されていることがわかる．

したがって，人間が特徴量を見て直感的に作成するランキングは，学習されるモデルによるランキングに近い結果を与えることが確かめられた．

### 補足: Footrule距離
ランキングの精度を確かめる一つの評価指標にSpearman's footruleが存在するため，作成したテストデータに対する評価を行う． Footrule距離は，次のように計算される．

$$
F(\sigma) = \sum_{i}\|i - \sigma(i)\|;
$$

4つの文書について考えられる2つの文書の組み合わせは，$_{4}\mathrm{C}_{2}=6$であり，
テストデータ1については，実際の順位と予測される適合度による順位が全て等しいため，Footrule距離は0となる一方で，
テストデータ２については，順序が真逆になるため，Footrule距離は$(\|1-4\|+\|2-3\|+\|3-2\|+\|4-1\|)=8$となる．

このように，ランキングに対する定量的な評価指標を利用することによっても，人間の直感によるランキングがモデルによるランキングに近しい結果を生み出すことを確認できる．